# Creating extensions using numpy and scipy

In this notebook, we shall go through two tasks:

1. Create a neural network layer with no parameters. 
  - This calls into **numpy** as part of it's implementation
2. Create a neural network layer that has learnable weights
  - This calls into **SciPy** as part of it's implementation

In [1]:
import torch
from torch.autograd import Function
from torch.autograd import Variable

## Parameter-less example

This layer doesn't particularly do anything useful or mathematically correct.

It is aptly named BadFFTFunction

**Layer Implementation**

In [2]:
from numpy.fft import rfft2, irfft2
class BadFFTFunction(Function):
    
    def forward(self, input):
        numpy_input = input.numpy()
        result = abs(rfft2(numpy_input))
        return torch.FloatTensor(result)
    
    def backward(self, grad_output):
        numpy_go = grad_output.numpy()
        result = irfft2(numpy_go)
        return torch.FloatTensor(result)

# since this layer does not have any parameters, we can
# simply declare this as a function, rather than as an nn.Module class
def incorrect_fft(input):
    return BadFFTFunction()(input)

**Example usage of the created layer:**

In [3]:
input = Variable(torch.randn(8, 8), requires_grad=True)
result = incorrect_fft(input)
print(result.data)
result.backward(torch.randn(result.size()))
print(input.grad)


  4.8235   2.8210   4.0698  11.3034   4.0292
  2.8678   6.5351   0.9228  15.3372   3.8725
  4.0342   7.6988   3.8099   4.0796  12.9163
  9.2561   6.0761   6.0133   5.7306   5.8949
  6.1931   3.0239   8.3571   9.1645   2.3575
  9.2561   7.6919   5.4074   9.5971   5.8949
  4.0342  13.2063   9.0728   5.2962  12.9163
  2.8678   2.4252   2.0834   4.9570   3.8725
[torch.FloatTensor of size 8x5]


 0.1849 -0.0055  0.0743 -0.0751  0.1089 -0.0751  0.0743 -0.0055
-0.0662  0.1506  0.1307 -0.0629 -0.1199  0.0800 -0.0873  0.1036
-0.0024 -0.0936  0.0083  0.0327 -0.1370 -0.2486 -0.0117 -0.0216
-0.0074 -0.1277  0.0631  0.0348  0.0422  0.1335  0.0221 -0.0900
 0.1353  0.0098  0.0030  0.0408 -0.0442  0.0408  0.0030  0.0098
-0.0074 -0.0900  0.0221  0.1335  0.0422  0.0348  0.0631 -0.1277
-0.0024 -0.0216 -0.0117 -0.2486 -0.1370  0.0327  0.0083 -0.0936
-0.0662  0.1036 -0.0873  0.0800 -0.1199 -0.0629  0.1307  0.1506
[torch.FloatTensor of size 8x8]



## Parametrized example

This implements a layer with learnable weights.

It implements the Cross-correlation with a learnable kernel.

In deep learning literature, it's confusingly referred to as Convolution.

The backward computes the gradients wrt the input and gradients wrt the filter.

**Implementation:**

*Please Note that the implementation serves as an illustration, and we did not verify it's correctness*

In [6]:
from scipy.signal import convolve2d, correlate2d
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter

class ScipyConv2dFunction(Function):
    
    def forward(self, input, filter):
        result = correlate2d(input.numpy(), filter.numpy(), mode='valid')
        self.save_for_backward(input, filter)
        return torch.FloatTensor(result)
    
    def backward(self, grad_output):
        input, filter = self.saved_tensors
        grad_input = convolve2d(grad_output.numpy(), filter.t().numpy(), mode='full')
        grad_filter = convolve2d(input.numpy(), grad_output.numpy(), mode='valid')
        return torch.FloatTensor(grad_input), torch.FloatTensor(grad_filter)


class ScipyConv2d(Module):
    
    def __init__(self, kh, kw):
        super(ScipyConv2d, self).__init__()
        self.filter=Parameter(torch.randn(kh, kw))
    
    def forward(self, input):
        return ScipyConv2dFunction()(input, self.filter)

**Example usage: **

In [7]:
module = ScipyConv2d(3, 3)
print(list(module.parameters()))
input = Variable(torch.randn(10, 10), requires_grad=True)
output = module(input)
print(output)
output.backward(torch.randn(8, 8))
print(input.grad)

[Parameter containing:
-0.1271  0.8109  0.4178
-0.5126 -1.1835 -0.2776
 0.4214  0.0886  1.0216
[torch.FloatTensor of size 3x3]
]
Variable containing:
 4.9120  0.6210  1.7908 -0.6933  3.4223  0.4025  2.0330  1.9110
-1.6563 -0.9113 -2.3579  0.5714 -2.4509 -1.1477 -2.2362  0.2235
-2.5879 -0.2629  0.0876  1.1707  1.2481  1.6186  1.2425  3.4960
 2.6881  2.0360  1.5574 -0.8602 -2.8442 -2.2571 -1.5803 -2.0943
-1.0176 -2.6565 -0.5091  1.5564  1.4575  3.1081  0.9570  1.0759
-2.7108  0.7214 -3.5160 -2.0563  0.6138 -2.6700 -1.1769 -1.0721
 2.8449  3.2918  1.6901  1.5427  2.4700  0.3433  2.2472  1.3501
-3.9733 -1.4927 -0.6596 -1.2467 -2.1322  1.1351 -1.4640 -0.5982
[torch.FloatTensor of size 8x8]


-0.0466 -0.1651  0.2731  0.0436 -0.1487 -0.3089  0.7800  0.4718  0.1200 -0.5005
 0.1600 -0.8988  1.6481  0.3330  0.3586 -2.7015  0.7774 -0.2702  1.4118  0.0614
 1.1303 -2.6691  0.4635  1.2966  2.5482 -3.1470  2.8663 -1.8794 -1.9309 -0.8698
-0.0614  1.5925 -0.7043 -0.9832 -0.7737 -4.6351  5.2933  0.2257 